# 11で出した重要語と共起する単語を算出

## 必要なもの
* ID, POPULATION_IDのセット
* local_stopwordsのセット

In [32]:
ID = "pikachu"

POPULATION_ID = "pokemon"

#10の途中経過
without_stopwords_text_file = "Progresses/NonStopword/%s-from-%s.txt"%(ID, POPULATION_ID)

#10の結果
sentiment_file = "Sentiment/%s-from-%s.txt"%(ID, POPULATION_ID)

##TFIDFモデル
tfidf_model_file = "Models/Tfidf/%s-from-%s.model"%(ID, POPULATION_ID)

#内部的に品詞を区別する区切り文字
TOKEN_DIVIDER = "<334>"

In [33]:
#ここだけのストップワード
#投稿そのものを除外するのではなく、関連単語から除外する
local_stopwords = ["ピカチュウ", "br", "無い"]

## 読み込み

In [34]:
import pandas as pd
import seaborn as sns

In [35]:
df_main = pd.read_table(without_stopwords_text_file)

#本ノートでは生テキストではなく形態素解析後のテキストが渡される
def tokenize(text):
    return text.split(TOKEN_DIVIDER)

df_main.head(2)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment
0,2016/02/23|21/35/00,60432,7,初ポケモンがピカチュウバージョンでひいこら言いながら進めてたこと ピカチュウが主力なの...,*<334>初<334>ポケモン<334>が<334>ピカチュウ<334>バージョン-ver...,初<334>ポケモン<334>ピカチュウ<334>バージョン-version<334>事<3...,neg
1,2008/02/25|00/58/00,26640,154,ダークシンジとピカチュウが来たらいよいよもって 懐かしいコテ勢揃いだな,*<334>ダーク-外国<334>シンジ<334>と<334>ピカチュウ<334>が<334...,ダーク-外国<334>シンジ<334>ピカチュウ<334>懐かしい<334>小手<334>勢揃い,pos


In [36]:
df_sentiment = pd.read_table(sentiment_file)

df_sentiment.head(2)

,Token,TFIDF,Sentiment,Counts
0,ピカチュウ,0.131934,-0.033769,16524
1,無い,0.023863,-0.332221,4497


## 単純な共起回数

### 単語ベクトルの用意

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


In [39]:
vectorizer = CountVectorizer(tokenizer=tokenize, stop_words=local_stopwords, ngram_range=(1,1), min_df=20)
vectorizer = vectorizer.fit(df_main["disassembled_target"].values.astype('U'))

c:\Users\brigh\anaconda3\envs\jpnlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [40]:
#keywordに共起する回数の多いものを数える
def get_freq(keyword):
    #keywordを含む行を走査
    df_keyword = np.empty((0, df_main.shape[1]))
    for _, row in df_main.iterrows():
        if keyword in tokenize(row["disassembled_all"]):
            df_keyword = np.vstack([df_keyword, row.values])

    #pandas行列に
    df_keyword = pd.DataFrame(df_keyword, columns=df_main.columns)

    #単語ベクトルの回数数える
    freq_spycy = vectorizer.transform(df_keyword["disassembled_target"])
    freq_df = pd.DataFrame(freq_spycy.toarray(), columns=vectorizer.get_feature_names_out())
    freq_sum = freq_df.sum(axis=0)
    
    #単語名と結び付け
    freq_sum = freq_sum.sort_values(ascending=False)

    #ストップワードの削除
    for word in local_stopwords:
        try:
            freq_sum = freq_sum.drop(word)
        except:
            pass

    return freq_sum

get_freq("良い").head(5)

良い      4445
ポケモン    1948
サトシ     1248
事       1090
2        655
dtype: int64

In [41]:
df_sentiment["CoOccurre-FREQ"] = ""
for index, row in df_sentiment.iterrows():
    freq = get_freq(row["Token"])
    end = 6
    if len(freq) <= 1:
        end = 1
    elif len(freq) < 6:
        end = len(freq)

    if end != 1:
        cooc = ", ".join(list(freq.iloc[1:end].index))
        df_sentiment.at[index, "CoOccurre-FREQ"] = cooc

pd.set_option('display.max_rows', 100)
df_sentiment.head(100).style.background_gradient(cmap="vlag_r", axis=0, subset="Sentiment")

,Token,TFIDF,Sentiment,Counts,CoOccurre-FREQ
0,ピカチュウ,0.131934,-0.033769,16524,"サトシ, 良い, 事, 1, 2"
1,無い,0.023863,-0.332221,4497,"事, 良い, サトシ, アニメ-animation, ポッチャマ"
2,ポケモン,0.022629,-0.088448,3878,"サトシ, 良い, 事, 2, ポッチャマ"
3,サトシ,0.022071,0.058747,2315,"ポケモン, 光, 良い, 事, ポッチャマ"
4,良い,0.019599,0.004146,3377,"ポケモン, サトシ, 事, 2, 方"
5,事,0.014171,-0.086328,2421,"ポケモン, サトシ, 良い, 時, 光"
6,可愛い,0.011501,0.230769,1079,"ポケモン, 良い, サトシ, 事, ライチュウ"
7,電気,0.011012,-0.002577,1164,"ポケモン, タイプ-type, 事, 良い, 技"
8,ライチュウ,0.010042,-0.108545,866,"ポケモン, 進化, 事, 良い, 電気"
9,ｗ,0.009813,-0.064293,1011,"ポケモン, 良い, 事, サトシ, 時"


## 分析対象内

## 母集団に対するTFIDF

### 母集団TFIDFモデルをサルベージ

In [42]:
import pickle
from copy import deepcopy

with open(tfidf_model_file, "rb") as f:
    tfidf_model = pickle.load(f)

#対象（名詞・形容詞）を見せる用
tfidf_model_showing = deepcopy(tfidf_model)
tfidf_model_showing.ngram = (1,1)
tfidf_model_showing.tokenizer = tokenize

### TFIDFを計算

In [43]:
#keywordに共起する回数の多いものを数える
def get_tfidf(keyword):
    #keywordを含む行を走査
    df_keyword = np.empty((0, df_main.shape[1]))
    for _, row in df_main.iterrows():
        if keyword in tokenize(row["disassembled_all"]):
            df_keyword = np.vstack([df_keyword, row.values])

    #pandas行列に
    df_keyword = pd.DataFrame(df_keyword, columns=df_main.columns)

    if df_keyword.shape[0] == 0:
        return pd.Series()

    #TFIDF計算
    tfidf_spycy = tfidf_model_showing.transform(df_keyword["disassembled_target"])
    tfidf_df = pd.DataFrame(tfidf_spycy.toarray(), columns=tfidf_model_showing.get_feature_names_out())
    tfidf_mean = tfidf_df.mean(axis=0)
    
    #単語名と結び付け
    tfidf_mean = tfidf_mean.sort_values(ascending=False)

    #ストップワードの削除
    for word in local_stopwords:
        try:
            tfidf_mean = tfidf_mean.drop(word)
        except:
            pass

    return tfidf_mean

get_tfidf("良い").head(5)

良い            0.147888
ポケモン          0.033628
サトシ           0.025689
ブイ-buoy       0.023690
良い ブイ-buoy    0.023503
dtype: float64

In [44]:
df_sentiment["CoOccurre-TFIDF"] = ""
for index, row in df_sentiment.iterrows():
    freq = get_tfidf(row["Token"])
    end = 6
    if len(freq) <= 1:
        end = 1
    elif len(freq) < 6:
        end = len(freq)

    if end != 1:
        cooc = ", ".join(list(freq.iloc[1:end].index))
        df_sentiment.at[index, "CoOccurre-TFIDF"] = cooc

pd.set_option('display.max_rows', 200)
df_sentiment.head(200).style.background_gradient(cmap="vlag_r", axis=0, subset="Sentiment")

C:\Users\brigh\AppData\Local\Temp\ipykernel_34388\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_34388\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_34388\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_34388\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


,Token,TFIDF,Sentiment,Counts,CoOccurre-FREQ,CoOccurre-TFIDF
0,ピカチュウ,0.131934,-0.033769,16524,"サトシ, 良い, 事, 1, 2","ポケモン, 良い, 事, 可愛い, ライチュウ"
1,無い,0.023863,-0.332221,4497,"事, 良い, サトシ, アニメ-animation, ポッチャマ","事, 良い, サトシ, 無い ピカチュウ, 事 無い"
2,ポケモン,0.022629,-0.088448,3878,"サトシ, 良い, 事, 2, ポッチャマ","ポケモン ピカチュウ, サトシ, 良い, 事, 自分"
3,サトシ,0.022071,0.058747,2315,"ポケモン, 光, 良い, 事, ポッチャマ","ポケモン, 光, 良い, 事, タケシ"
4,良い,0.019599,0.004146,3377,"ポケモン, サトシ, 事, 2, 方","ポケモン, サトシ, ブイ-buoy, 良い ブイ-buoy, 事"
5,事,0.014171,-0.086328,2421,"ポケモン, サトシ, 良い, 時, 光","ポケモン, サトシ, 事 無い, 良い, 時"
6,可愛い,0.011501,0.230769,1079,"ポケモン, 良い, サトシ, 事, ライチュウ","ピカチュウ 可愛い, 可愛い ピカチュウ, ポケモン, 良い, ライチュウ"
7,電気,0.011012,-0.002577,1164,"ポケモン, タイプ-type, 事, 良い, 技","電気 玉, 玉, 電気 鼠, 鼠, 電気 だま"
8,ライチュウ,0.010042,-0.108545,866,"ポケモン, 進化, 事, 良い, 電気","進化, ピチュー, 電気, 可愛い, ポケモン"
9,ｗ,0.009813,-0.064293,1011,"ポケモン, 良い, 事, サトシ, 時","無い ｗ, ポケモン, 良い, サトシ, 事"
